# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
from pprint import pprint

#import API keys
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load the csv exported in Part I to a DataFrame
cityData = pd.read_csv('output_data\cities.csv')
cityData_pd = pd.DataFrame(cityData)

#clear old (identical) index column
cityData_pd = cityData_pd.drop(columns='Unnamed: 0')

cityData_pd.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,cape town,ZA,-33.9258,18.4232,57.99,94,0,3.44,1619211557
1,coruripe,BR,-10.1256,-36.1756,78.80,79,54,8.43,1619211557
2,adrar,MR,20.5022,-10.0711,83.97,8,0,7.74,1619211558
3,great yarmouth,GB,52.6083,1.7305,45.00,61,69,3.44,1619211558
4,hobart,AU,-42.8794,147.3294,51.80,71,40,10.36,1619211559


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [4]:
#use the Lat and Lng as locations and Humidity as the weight
locations = cityData_pd[["Lat", "Lng"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=cityData_pd['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

#add Heatmap layer to map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#my ideal weather conditions are:
#  - 68F <= Max Temp <= 85F
#  - Cloudiness <= 70%
#  - Humidity <= 50%   

hotel_df = cityData_pd.loc[(cityData_pd['Max Temp'] >= 68) & (cityData_pd['Max Temp'] <= 85) & 
                           (cityData_pd['Cloudiness'] <= 70) & (cityData_pd['Humidity'] <= 50), :]

#drop rows with nulls
hotel_df = hotel_df.dropna()


hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
2,adrar,MR,20.5022,-10.0711,83.97,8,0,7.74,1619211558
77,baghdad,IQ,33.3406,44.4009,78.80,27,0,6.91,1619211597
110,awbari,LY,26.5921,12.7805,84.94,8,6,9.60,1619211624
113,laguna,US,38.4210,-121.4238,75.99,45,1,11.50,1619211626
132,buraydah,SA,26.3260,43.9750,84.20,17,0,6.91,1619211524
137,viedma,AR,-40.8135,-62.9967,68.83,41,58,7.58,1619211641
164,roebourne,AU,-20.7833,117.1333,75.33,26,0,11.50,1619211660
176,umm lajj,SA,25.0213,37.2685,84.54,42,4,5.26,1619211668
239,port hedland,AU,-20.3167,118.5667,69.80,37,0,8.05,1619211704
268,beni suef,EG,29.0731,31.0979,69.01,40,0,14.97,1619211719


In [22]:
#add new blank column
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() 
                                  + ['Hotel Name'])
hotel_df['Hotel Name'] = ''

#reset index to be neat
hotel_df.reset_index(drop=True, inplace=True)

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,adrar,MR,20.5022,-10.0711,83.97,8,0,7.74,1619211558,
1,baghdad,IQ,33.3406,44.4009,78.80,27,0,6.91,1619211597,
2,awbari,LY,26.5921,12.7805,84.94,8,6,9.60,1619211624,
3,laguna,US,38.4210,-121.4238,75.99,45,1,11.50,1619211626,
4,buraydah,SA,26.3260,43.9750,84.20,17,0,6.91,1619211524,
5,viedma,AR,-40.8135,-62.9967,68.83,41,58,7.58,1619211641,
6,roebourne,AU,-20.7833,117.1333,75.33,26,0,11.50,1619211660,
7,umm lajj,SA,25.0213,37.2685,84.54,42,4,5.26,1619211668,
8,port hedland,AU,-20.3167,118.5667,69.80,37,0,8.05,1619211704,
9,beni suef,EG,29.0731,31.0979,69.01,40,0,14.97,1619211719,


In [23]:
#google places api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_names = []
row_index = 0

#go through rows making successive api calls and appending hotel name to list
for row in hotel_df.iterrows():
    
    try:
        location = str(hotel_df.at[row_index, 'Lat']) + ',' + str(hotel_df.at[row_index,'Lng'])       
        params = {
            "location": location,
            "radius": 5000,
            "type": "lodging",
            "key": g_key,
            }
        row_index += 1
        response = requests.get(base_url, params=params).json()
        hotel_names.append(response['results'][0]['name'])
    
    
    except:
        hotel_names.append('No hotels found')

#import hotel name list into Hotel Name column in DataFrame         
hotel_df['Hotel Name'] = hotel_names

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,adrar,MR,20.5022,-10.0711,83.97,8,0,7.74,1619211558,No hotels found
1,baghdad,IQ,33.3406,44.4009,78.80,27,0,6.91,1619211597,Baghdad Hotel
2,awbari,LY,26.5921,12.7805,84.94,8,6,9.60,1619211624,فندق أوباري
3,laguna,US,38.4210,-121.4238,75.99,45,1,11.50,1619211626,Holiday Inn Express & Suites Elk Grove Central...
4,buraydah,SA,26.3260,43.9750,84.20,17,0,6.91,1619211524,فندق راحة السلام
5,viedma,AR,-40.8135,-62.9967,68.83,41,58,7.58,1619211641,Hotel Cristal
6,roebourne,AU,-20.7833,117.1333,75.33,26,0,11.50,1619211660,Harding River Caravan Park
7,umm lajj,SA,25.0213,37.2685,84.54,42,4,5.26,1619211668,Moon light Furnished Units
8,port hedland,AU,-20.3167,118.5667,69.80,37,0,8.05,1619211704,The Esplanade Hotel
9,beni suef,EG,29.0731,31.0979,69.01,40,0,14.97,1619211719,Jewel Inn Beni Suef


In [24]:
#NOTE: Do not change any of the code in this cell

#using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#store the DataFrame Row


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
#add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info )

fig.add_layer(marker_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))